### Bidirectional LSTM

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df=pd.read_csv("train.csv")

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
df.shape

(20800, 5)

## Drop Nan Values

In [7]:
df=df.dropna()

In [9]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [10]:
df.shape

(18285, 5)

## Get the Independent Features

In [12]:
X=df.drop("label",axis=1)

##  Get the Independent Features

In [13]:
y=df["label"]

In [17]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [18]:
import tensorflow as tf

In [19]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence  import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional


## Vocabulary size


In [20]:
voc_size=5000

In [21]:
messages=X.copy()

In [22]:
messages.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [23]:
messages["title"][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

## only we r going to work on title column (one hot encoding)
## messages.reset_index(inplace=True) not important

In [25]:
messages.reset_index(inplace=True)

In [26]:
messages.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [28]:
import nltk
import re
from nltk.corpus import stopwords


In [29]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Afzal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Dataset Preprocessing

In [30]:
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()

In [37]:
corpus=[]

for i in range(len(messages)):
    review=re.sub("[^a-zA-Z]"," ",messages["title"][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(words)for words in review if  not words in stopwords.words("english")]
    review = ' '.join(review)
    corpus.append(review)
    
    
   
    
    



In [38]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

## Onehot Representation

In [39]:
onehot_repr=[one_hot (words,voc_size)for words in corpus]
onehot_repr

[[2520, 4692, 3974, 887, 851, 228, 2884, 1057, 1800, 3929],
 [3927, 653, 304, 123, 3265, 1959, 3348],
 [2704, 2217, 2824, 2605],
 [2466, 3877, 2011, 3668, 1283, 4889],
 [205, 3265, 3612, 4269, 3868, 1002, 3265, 2845, 3636, 32],
 [3053,
  4113,
  209,
  4275,
  4563,
  1629,
  2730,
  3706,
  1482,
  1076,
  1640,
  4064,
  3168,
  4228,
  3348],
 [3933, 4638, 4786, 1370, 19, 1229, 1085, 1692, 395, 70, 4968],
 [1702, 2163, 4422, 3323, 4802, 902, 1629, 2554, 395, 70, 4968],
 [2, 1063, 2995, 284, 2613, 3698, 340, 215, 1629, 3169],
 [2021, 888, 2448, 4755, 3392, 4558, 3126, 3603],
 [2381, 2769, 3086, 502, 1322, 4112, 2007, 3016, 1492, 51, 3847],
 [3668, 4562, 851, 3698, 1629, 4802],
 [2393, 4978, 1124, 2975, 1473, 892, 4877, 386, 210],
 [1712, 872, 836, 3674, 1866, 712, 4766, 395, 70, 4968],
 [4531, 1025, 2218, 3039, 23, 395, 70, 4968],
 [1, 417, 3637, 1957, 3076, 4694, 2136, 4898, 2770, 1356],
 [1473, 2585, 653],
 [865, 1098, 895, 2092, 1629, 2538, 2652, 3348],
 [1950, 4487, 304, 4999, 32

In [40]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [41]:
onehot_repr[1]

[3927, 653, 304, 123, 3265, 1959, 3348]

## Embedding Representation
## all the sentence has diffrent length so we have to make equal size
## so the labirary w r going to use padding

In [42]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[2520 4692 3974 ...    0    0    0]
 [3927  653  304 ...    0    0    0]
 [2704 2217 2824 ...    0    0    0]
 ...
 [ 978 2777 3504 ...    0    0    0]
 [2964 4802 2931 ...    0    0    0]
 [4280 3132 2312 ...    0    0    0]]


## Creating model

In [44]:
embedding_vector_features=40 ##features representation (every index will be represented by 40 feture with some diffrent diffrent values )
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))    # neuron 
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [45]:
len(embedded_docs),y.shape

(18285, (18285,))

In [46]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [47]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

# now split data into training and test¶

In [48]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

## Model Training

In [49]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 26s 81ms/step - loss: 0.3027 - accuracy: 0.8572 - val_loss: 0.1868 - val_accuracy: 0.9234
Epoch 2/10
192/192 [==============================] - 14s 71ms/step - loss: 0.1264 - accuracy: 0.9527 - val_loss: 0.1878 - val_accuracy: 0.9185
Epoch 3/10
192/192 [==============================] - 14s 75ms/step - loss: 0.0845 - accuracy: 0.9714 - val_loss: 0.2159 - val_accuracy: 0.9153
Epoch 4/10
192/192 [==============================] - 16s 81ms/step - loss: 0.0586 - accuracy: 0.9812 - val_loss: 0.2316 - val_accuracy: 0.9168
Epoch 5/10
192/192 [==============================] - 18s 95ms/step - loss: 0.0399 - accuracy: 0.9887 - val_loss: 0.2999 - val_accuracy: 0.9122
Epoch 6/10
192/192 [==============================] - 16s 86ms/step - loss: 0.0256 - accuracy: 0.9930 - val_loss: 0.3695 - val_accuracy: 0.9115
Epoch 7/10
192/192 [==============================] - 14s 70ms/step - loss: 0.0204 - accuracy: 0.9944 - val_loss: 0.4304 - val_accuracy:

## Performance Metrics And Accuracy

In [50]:
y_pred=model.predict(X_test)

189/189 [==============================] - 5s 14ms/step


In [51]:
y_pred=np.where(y_pred > 0.5, 1,0)

In [54]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [55]:
confusion_matrix(y_test,y_pred)

array([[3110,  309],
       [ 245, 2371]], dtype=int64)

In [56]:
accuracy_score(y_test,y_pred)

0.9082021541010771

In [57]:

from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))


              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.88      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

